In [264]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt


df = pd.read_csv('clean_dataset.csv')

# Configure data type to String
df['Year End'] = df['Year End'].astype(str)

# List numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns

for column in numeric_cols:   
    sns.boxplot(data=df, x='Year End', y=column)
    plt.show()

These columns had very obvious outliers:

gross combination ratio, gross expense ratio, net combination ratio, net expense ratio, pure gross claims ratio, pure net claims ratio, SCR Coverage ratio

The data distribution for all columns also seem to be skewed (mostly to the right). Natural to have few big players and many small players in the insurance industry. 

Because it is a skewed distribution, I will use the interquartile range to identify them.

Since it is hard to differentiate between outlier and we want to find the big and risky firms to supervise, I will be very conservative and allow for a very high upper limit

In [265]:
# Calculate quantiles and IQR
Q1 = df[numeric_cols].quantile(0.25) 
Q3 = df[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

upper_limit = Q3 + 10 * IQR
lower_limit = Q1 - 3 * IQR

# Return a boolean array of the rows with (any) non-outlier column values
condition = ~((df[numeric_cols] < (lower_limit)) | (df[numeric_cols] > (upper_limit))).any(axis=1)

# Create a dataframe of rows which fall outside limits
df_within_limit = df[condition]

df_outside_limit = df[~condition]

# Update conditions for firms which have have more than two rows outside limit:
for i, firm_names in df_outside_limit['Firm Name'].items():
    if len(df_outside_limit[df_outside_limit['Firm Name'] == firm_names]) > 1:
        condition[i]=True

# New upper limit
new_upper_limit = Q3 + 50 * IQR
new_lower_limit = Q1 - 3 * IQR

# Return a boolean array of the rows with (any) non-outlier column values
condition = ~((df[numeric_cols] < (new_lower_limit)) | (df[numeric_cols] > (new_upper_limit))).any(axis=1)


# Filter our dataframe based on condition
df_no_outlier = df[condition]

In [267]:
%%HTML
<div class='tableauPlaceholder' id='viz1701026923696' style='position: relative'><noscript><a href='#'><img alt='BoE Supervision Challenge ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Bo&#47;BoESupervisionChallenge-NewDistribution&#47;DistributionDashboard&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='BoESupervisionChallenge-NewDistribution&#47;DistributionDashboard' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Bo&#47;BoESupervisionChallenge-NewDistribution&#47;DistributionDashboard&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-GB' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1701026923696');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1000px';vizElement.style.height='827px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1000px';vizElement.style.height='827px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [268]:
df_no_outlier.to_csv('clean_dataset_2.csv', encoding='utf-8', index=False)

rows removed as a result of trimming:

2280-2101 = 27